In [5]:
#brown corpus
import nltk
from nltk.corpus import brown
print brown.categories()
print brown.sents(categories='news')

[u'adventure', u'belles_lettres', u'editorial', u'fiction', u'government', u'hobbies', u'humor', u'learned', u'lore', u'mystery', u'news', u'religion', u'reviews', u'romance', u'science_fiction']
[[u'The', u'Fulton', u'County', u'Grand', u'Jury', u'said', u'Friday', u'an', u'investigation', u'of', u"Atlanta's", u'recent', u'primary', u'election', u'produced', u'``', u'no', u'evidence', u"''", u'that', u'any', u'irregularities', u'took', u'place', u'.'], [u'The', u'jury', u'further', u'said', u'in', u'term-end', u'presentments', u'that', u'the', u'City', u'Executive', u'Committee', u',', u'which', u'had', u'over-all', u'charge', u'of', u'the', u'election', u',', u'``', u'deserves', u'the', u'praise', u'and', u'thanks', u'of', u'the', u'City', u'of', u'Atlanta', u"''", u'for', u'the', u'manner', u'in', u'which', u'the', u'election', u'was', u'conducted', u'.'], ...]


In [9]:
cfd = nltk.ConditionalFreqDist( (genre, word)
    for genre in brown.categories()
    for word in brown.words(categories=genre))

genres = ['news', 'religion', 'hobbies', 'science_fiction', 'romance', 'humor'] 
modals = ['can', 'could', 'may', 'might', 'must', 'will'] 
cfd.tabulate(conditions=genres, samples=modals)

                  can could   may might  must  will 
           news    93    86    66    38    50   389 
       religion    82    59    78    12    54    71 
        hobbies   268    58   131    22    83   264 
science_fiction    16    49     4    12     8    16 
        romance    74   193    11    51    45    43 
          humor    16    30     8     8     9    13 


In [ ]:
#loading own corpus
from nltk.corpus import PlaintextCorpusReader
corpus_root = path_file
wordLists = PlaintextCorpusReader(corpus_root, '.*')
wordLists.words(wordLists.fileids()[0])

In [13]:
#bigrams
sent = ['In', 'the', 'beginning', 'God', 'created', 'the', 'heaven', 'and', 'the', 'earth', '.']
list(nltk.bigrams(sent))

[('In', 'the'),
 ('the', 'beginning'),
 ('beginning', 'God'),
 ('God', 'created'),
 ('created', 'the'),
 ('the', 'heaven'),
 ('heaven', 'and'),
 ('and', 'the'),
 ('the', 'earth'),
 ('earth', '.')]

In [15]:
#proportion of non-stopping words
def content_fraction(text):
    stopwords = nltk.corpus.stopwords.words('english')
    content = [w for w in text if w.lower() not in stopwords]
    return 1.0 * len(content) / len(text)

content_fraction(nltk.corpus.reuters.words())

0.735240435097661

In [16]:
#pronoucing dictionary
entries = nltk.corpus.cmudict.entries()
for entry in entries[39943:39951]:
    print entry

(u'explorer', [u'IH0', u'K', u'S', u'P', u'L', u'AO1', u'R', u'ER0'])
(u'explorers', [u'IH0', u'K', u'S', u'P', u'L', u'AO1', u'R', u'ER0', u'Z'])
(u'explores', [u'IH0', u'K', u'S', u'P', u'L', u'AO1', u'R', u'Z'])
(u'exploring', [u'IH0', u'K', u'S', u'P', u'L', u'AO1', u'R', u'IH0', u'NG'])
(u'explosion', [u'IH0', u'K', u'S', u'P', u'L', u'OW1', u'ZH', u'AH0', u'N'])
(u'explosions', [u'IH0', u'K', u'S', u'P', u'L', u'OW1', u'ZH', u'AH0', u'N', u'Z'])
(u'explosive', [u'IH0', u'K', u'S', u'P', u'L', u'OW1', u'S', u'IH0', u'V'])
(u'explosively', [u'EH2', u'K', u'S', u'P', u'L', u'OW1', u'S', u'IH0', u'V', u'L', u'IY0'])


In [17]:
#find words with similar pronounciation
for word, pron in entries:
    if len(pron) == 3:
        p1, p2, p3 = pron
        if p1 == 'P' and p3 == 'T':
            print word

pait
pat
pate
patt
peart
peat
peet
peete
pert
pet
pete
pett
piet
piette
pit
pitt
pot
pote
pott
pout
puett
purt
put
putt


In [19]:
from nltk.corpus import toolbox
#toolbox.entries('rotokas.dic')

In [31]:
#wordnet synonyms
from nltk.corpus import wordnet as wn
wn.synsets('motorcar')

[Synset('car.n.01')]

In [26]:
wn.synset('car.n.01').lemma_names()

[u'car', u'auto', u'automobile', u'machine', u'motorcar']

In [29]:
#look at concepts that're more specific
wn.synset('car.n.01').hyponyms()[:5]

[Synset('ambulance.n.01'),
 Synset('beach_wagon.n.01'),
 Synset('bus.n.04'),
 Synset('cab.n.03'),
 Synset('compact.n.03')]

In [34]:
#look at concepts that're more general
wn.synset('car.n.01').hypernym_paths()[0]

[Synset('entity.n.01'),
 Synset('physical_entity.n.01'),
 Synset('object.n.01'),
 Synset('whole.n.02'),
 Synset('artifact.n.01'),
 Synset('instrumentality.n.03'),
 Synset('container.n.01'),
 Synset('wheeled_vehicle.n.01'),
 Synset('self-propelled_vehicle.n.01'),
 Synset('motor_vehicle.n.01'),
 Synset('car.n.01')]

In [35]:
#the `include` relationship
wn.synset('tree.n.01').part_meronyms()

[Synset('burl.n.02'),
 Synset('crown.n.07'),
 Synset('limb.n.02'),
 Synset('stump.n.01'),
 Synset('trunk.n.01')]

In [36]:
#semantic similarity of two words
right = wn.synset('right_whale.n.01') 
orca = wn.synset('orca.n.01')
minke = wn.synset('minke_whale.n.01')

In [40]:
#higher min_depth, more similar
right.lowest_common_hypernyms(minke)[0].min_depth()

14

# Processing Raw Text

In [42]:
#tokenizing
from urllib import urlopen
url = "http://www.gutenberg.org/files/2554/2554.txt"
raw = urlopen(url).read()

tokens = nltk.word_tokenize(raw)
tokens[:10]

['The',
 'Project',
 'Gutenberg',
 'EBook',
 'of',
 'Crime',
 'and',
 'Punishment',
 ',',
 'by']

In [50]:
#tokens to nltk text
text = nltk.Text(tokens)
text.vocab()

FreqDist({'fawn': 1,
          'writings': 1,
          'nunnery': 1,
          'yellow': 27,
          'four': 26,
          'Does': 8,
          'clotted': 1,
          'spiders': 2,
          'hanging': 15,
          'self-reliant': 1,
          'Foundation': 24,
          'Livingstone': 1,
          '_might_': 1,
          'rupture': 7,
          'rouse': 4,
          'Fetch': 1,
          'originality': 1,
          'superficially': 1,
          'unnecessarily': 1,
          'meadows': 1,
          'sinking': 4,
          'impertinent': 1,
          'fogeys': 1,
          'uncertain': 5,
          'stabbed': 5,
          'bringing': 14,
          'disturb': 6,
          'recollections': 2,
          'uttering': 12,
          'prize': 2,
          'buttonhole': 1,
          'protest': 14,
          'persisted': 8,
          's/he': 1,
          'Evidently': 2,
          'specially': 2,
          'tired': 17,
          'railing': 5,
          'pulse': 3,
          'elegant': 2,
    

In [51]:
#words commonly appear together
text.collocations()

Katerina Ivanovna; Pyotr Petrovitch; Pulcheria Alexandrovna; Avdotya
Romanovna; Rodion Romanovitch; Marfa Petrovna; Sofya Semyonovna; old
woman; Project Gutenberg-tm; Porfiry Petrovitch; Amalia Ivanovna;
great deal; Nikodim Fomitch; young man; Ilya Petrovitch; n't know;
Project Gutenberg; Dmitri Prokofitch; Andrey Semyonovitch; Hay Market


In [ ]:
#sentence segmentation
sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
text = nltk.corpus.gutenberg.raw('some_file')
sens = sent_tokenizer(text)

In [ ]:
P201